In [6]:
import yfinance as yf
import google.generativeai as genai
import smtplib
import time
import re
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from jinja2 import Template
from google.colab import userdata

# --- 1. SECURE CONFIGURATION ---
api_key = userdata.get('GOOGLE_API_KEY')
sender_email = userdata.get('EMAIL_USER')
app_password = userdata.get('EMAIL_PASS')

genai.configure(api_key=api_key)

# List available models and choose one supported for 'generateContent'
print("Listing available models...")
chosen_model_name = None
all_available_models = genai.list_models()

# Filter for models that support 'generateContent'
supported_generation_models = [m.name for m in all_available_models if 'generateContent' in m.supported_generation_methods]

# Prioritize specific models
if 'gemini-pro' in supported_generation_models:
    chosen_model_name = 'gemini-pro'
    print("Using gemini-pro model.")
elif 'gemini-1.0-pro' in supported_generation_models:
    chosen_model_name = 'gemini-1.0-pro'
    print("Using gemini-1.0-pro model as fallback.")
elif 'gemini-pro-latest' in supported_generation_models:
    chosen_model_name = 'gemini-pro-latest'
    print("Using gemini-pro-latest model.")
elif 'gemini-flash-latest' in supported_generation_models:
    chosen_model_name = 'gemini-flash-latest'
    print("Using gemini-flash-latest model.")
elif supported_generation_models:
    # Fallback to the first available model that supports generateContent
    chosen_model_name = supported_generation_models[0]
    print(f"No preferred model found. Using first available: {chosen_model_name}")
else:
    print("No models supporting 'generateContent' found at all. Defaulting to 'gemini-pro' (will likely fail).")
    chosen_model_name = 'gemini-pro' # Absolute default, likely to fail if no models are supported

model = genai.GenerativeModel(chosen_model_name)

# Market Hunting List
HUNTING_LIST = ["ATD.TO", "DOL.TO", "L.TO", "MRU.TO", "COST", "PG", "KO", "LULU", "WMT", "NKE"]

# --- 2. IMPROVED JINJA2 TEMPLATE (High Contrast & Clean) ---
EMAIL_TEMPLATE = """
<html>
<head>
<style>
    body { font-family: 'Segoe UI', Arial, sans-serif; color: #333; background-color: #f4f4f4; margin: 0; padding: 20px; }
    .report-card { border: 1px solid #ddd; padding: 25px; margin-bottom: 30px; border-radius: 12px; background: white; box-shadow: 0 4px 8px rgba(0,0,0,0.05); }

    /* FIX: Force white text for dark green background headers */
    .header { background-color: #1a472a; padding: 15px; border-radius: 8px 8px 0 0; margin: -25px -25px 20px -25px; }
    .header h2 { color: #ffffff !important; margin: 0; font-size: 22px; font-weight: bold; }

    table { width: 100%; border-collapse: collapse; margin: 20px 0; background: #fff; }
    th, td { border: 1px solid #eee; padding: 12px; text-align: left; }
    th { background-color: #f8f9fa; color: #1a472a; font-weight: bold; text-transform: uppercase; font-size: 12px; }

    .section-title { color: #1a472a; border-bottom: 2px solid #1a472a; padding-bottom: 5px; margin-top: 25px; font-size: 18px; }
    .verdict { font-size: 1.3em; font-weight: bold; padding: 10px; border-radius: 5px; display: inline-block; margin-top: 10px; }
    .buy { color: #28a745; background: #eef9f1; }
</style>
</head>
<body>
    <div class="report-card">
        <div class="header"><h2>{{ ticker }} Analysis - {{ date }}</h2></div>
        {{ ai_content | safe }}
        <p style="font-size: 11px; color: #888; border-top: 1px solid #eee; padding-top: 10px; margin-top: 30px;">
            Confidential TFSA Analyst Report. Prepared for personal use in Vancouver.
        </p>
    </div>
</body>
</html>
"""

# --- 3. CORE LOGIC FUNCTIONS ---

def clean_llm_output(text):
    """Removes all Markdown formatting so only raw HTML remains."""
    # Handle None input explicitly
    if text is None:
        return "<p>LLM did not return any text content.</p>"
    # Remove markdown code blocks (```html or ```)
    text = re.sub(r'```html', '', text, flags=re.IGNORECASE)
    text = re.sub(r'```', '', text)
    # Remove any stray markdown headers (###) or bolding (**)
    text = re.sub(r'###\s+', '<h3 class="section-title">', text)
    text = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', text)
    return text.strip()

def get_buffett_analysis(ticker, user_question=None):
    """Fetches stock data and generates a structured, organized English report."""
    system_instr = """
    You are a Senior Investment Analyst.
    1. Respond in PROFESSIONAL ENGLISH ONLY.
    2. DO NOT use Chinese translations.
    3. DO NOT use markdown (no ###, no **).
    4. Use raw HTML: <table> for metrics, <h3 class="section-title"> for sections.
    5. State the Verdict clearly at the end.
    """

    metrics = None
    if ticker: # Only attempt to fetch data if a ticker is actually provided
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            if info and info.get('currentPrice'): # Ensure some basic info is present
                metrics = {
                    "Ticker": ticker,
                    "Price": f"{info.get('currentPrice')} {info.get('currency')}",
                    "ROE": f"{info.get('returnOnEquity', 0):.2%}",
                    "PE": info.get('forwardPE'),
                    "DE": info.get('debtToEquity', 0) / 100
                }
        except Exception as e:
            # If yfinance fails for a *real* ticker, report it.
            return f"Error: Could not retrieve valid stock data for {ticker}. Please check the ticker symbol. ({e})"

    # Now construct the prompt based on whether we have metrics and a user_question
    if user_question:
        if metrics:
            # Question about a specific stock with metrics
            prompt = f"{system_instr}\n\nUser Question: {user_question}\nReference Stock: {ticker}\nReference Metrics: {metrics}"
        else:
            # General question, or question about a stock for which we couldn't get metrics
            prompt = f"{system_instr}\n\nUser Question: {user_question}\n"
            if ticker: # If a ticker was provided but no metrics, inform the LLM
                prompt += f"(Note: No valid stock data could be retrieved for {ticker}.)"
        try:
            response = model.generate_content(prompt)
            # Ensure response.text is not None before cleaning
            if response and hasattr(response, 'text') and response.text is not None:
                return clean_llm_output(response.text)
            else:
                return "<p>Error: Gemini API did not return text content for this query.</p>"
        except Exception as e:
            return f"<p>Error during AI generation or cleaning: {e}</p>"
    elif metrics:
        # Direct stock analysis request (no user_question)
        prompt = f"{system_instr}\n\nPerform a Buffett-style analysis for {ticker} using {metrics}. Include a Verdict (Buy/Wait)."
        try:
            response = model.generate_content(prompt)
            # Ensure response.text is not None before cleaning
            if response and hasattr(response, 'text') and response.text is not None:
                return clean_llm_output(response.text)
            else:
                return f"<p>Error: Gemini API did not return text content for {ticker} analysis.</p>"
        except Exception as e:
            return f"<p>Error during AI generation or cleaning for {ticker}: {e}</p>"
    else:
        return "Please provide a stock ticker to analyze or ask a question."

def send_email(html_body, subject="Buffett Strategic Alert"):
    """Sends the formatted HTML report to your inbox."""
    msg = MIMEMultipart("alternative")
    msg["Subject"] = f"{subject} ({datetime.now().strftime('%Y-%m-%d')})"
    msg["From"] = sender_email
    msg["To"] = sender_email
    msg.attach(MIMEText(html_body, "html"))

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.sendmail(sender_email, sender_email, msg.as_string())

# --- 4. THE INTEGRATED EXECUTION ---

def run_agent_system():
    # PART A: THE AUTOMATIC MARKET SCAN
    print(f"🚀 Starting market scan of {len(HUNTING_LIST)} stocks...")
    scan_reports = ""
    for ticker in HUNTING_LIST:
        print(f"🔍 Analyzing {ticker}...")
        report = get_buffett_analysis(ticker)

        # Only email high-conviction 'Buy' signals
        if "Buy" in report:
            print(f"✅ BUY SIGNAL: {ticker}")
            tm = Template(EMAIL_TEMPLATE)
            scan_reports += tm.render(ticker=ticker, date=datetime.now().strftime("%Y-%m-%d"), ai_content=report) + "<hr>"

        time.sleep(12) # Respect 5 RPM limit

    if scan_reports:
        send_email(scan_reports, subject="3-Day Market Scan Report")
        print("📧 Scheduled Scan Email Sent.")

    # PART B: THE INTERACTIVE CHAT BOX
    print("\n" + "="*50)
    print("🤖 INTERACTIVE ANALYST ACTIVE")
    print("Ask follow-up questions about a stock (e.g., 'What about AAPL?') or 'exit' to quit.")
    print("To analyze a new stock, just type its ticker.")

    last_analyzed_ticker = None

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['exit', 'quit']: break

        # Try to identify if the input is a ticker or a question
        # Simple heuristic: if it's all caps and less than 10 chars, treat as ticker
        if user_input.strip().isalpha() and len(user_input.strip()) <= 10 and user_input.strip().upper() == user_input.strip():
            ticker_to_analyze = user_input.strip().upper()
            last_analyzed_ticker = ticker_to_analyze
            print(f"🔍 Analyzing {ticker_to_analyze}...")
            chat_report = get_buffett_analysis(ticker_to_analyze)
        else:
            # It's a question. If there's a last_analyzed_ticker, use it.
            if last_analyzed_ticker:
                print(f"❓ Answering question about {last_analyzed_ticker}...")
                chat_report = get_buffett_analysis(last_analyzed_ticker, user_question=user_input)
            else:
                # No ticker context, ask the LLM to answer generally
                print("❓ Answering general question...")
                chat_report = get_buffett_analysis(None, user_question=user_input) # Pass None for ticker if no context

        # Display directly in Colab
        from IPython.display import HTML, display
        display(HTML(chat_report))

        if input("Send this chat result to your email? (y/n): ").lower() == 'y':
            tm = Template(EMAIL_TEMPLATE)
            # Use the actual ticker for email subject if available, otherwise 'Interactive Query'
            email_ticker_name = last_analyzed_ticker if last_analyzed_ticker else "Interactive Query"
            final_html = tm.render(ticker=email_ticker_name, date=datetime.now().strftime("%Y-%m-%d"), ai_content=chat_report)
            send_email(final_html, subject=f"Interactive Analysis Result for {email_ticker_name}")

# Start the system
run_agent_system()

Listing available models...
No preferred model found. Using first available: models/gemini-2.5-flash
🚀 Starting market scan of 10 stocks...
🔍 Analyzing ATD.TO...
✅ BUY SIGNAL: ATD.TO
🔍 Analyzing DOL.TO...
🔍 Analyzing L.TO...
🔍 Analyzing MRU.TO...
🔍 Analyzing COST...
🔍 Analyzing PG...
🔍 Analyzing KO...
✅ BUY SIGNAL: KO
🔍 Analyzing LULU...
✅ BUY SIGNAL: LULU
🔍 Analyzing WMT...
🔍 Analyzing NKE...
📧 Scheduled Scan Email Sent.

🤖 INTERACTIVE ANALYST ACTIVE
Ask follow-up questions about a stock (e.g., 'What about AAPL?') or 'exit' to quit.
To analyze a new stock, just type its ticker.

You: Unilever
❓ Answering general question...


Metric,Value (Illustrative of Recent Performance)
Revenue (FY2023),€60.1 Billion
Operating Profit (FY2023),€9.9 Billion
Net Profit (FY2023),€7.1 Billion
Diluted EPS (FY2023),€2.87
Free Cash Flow (FY2023),€7.0 Billion
Dividend Per Share (FY2023),€1.79
Dividend Yield (Approx.),3.5%
Gross Margin (FY2023),42.2%
Operating Margin (FY2023),16.5%
Net Debt (FY2023),€23.0 Billion


Send this chat result to your email? (y/n): y

You: exit
